## HW3 Image Classification
#### Solve image classification with convolutional neural networks(CNN).
#### If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

In [ ]:
# check GPU type.
!nvidia-smi

### Import Packages

In [ ]:
_exp_name = "sample"

In [ ]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import Dataset, DataLoader, Subset, TensorDataset, random_split, SubsetRandomSampler, ConcatDataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [ ]:
myseed = 1213  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

### Transforms

In [ ]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Posibility of transforms
tfm_p = 0.3

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.RandomAutocontrast(p=tfm_p),
    transforms.RandomEqualize(p=tfm_p),
    transforms.RandomPerspective(p=tfm_p),
    transforms.RandomVerticalFlip(p=tfm_p),
    transforms.RandomHorizontalFlip(p=tfm_p),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

### Datasets

In [ ]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        if files != None:
            self.files = files
            
        self.transform = tfm
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        im = self.transform(im)
        
        try:
            label = int(fname.split("/")[-1].split("_")[0])
        except:
            label = -1 # test has no label
            
        return im,label

### Model

In [ ]:
# Posibility of dropout layers
dropout_layer_p = 0.2

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]
            
            
            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]
            
            nn.Conv2d(256, 384, 3, 1, 1), # [384, 16, 16]
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [384, 8, 8]
            
            nn.Conv2d(384, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_layer_p),
        
            nn.Conv2d(512, 640, 3, 1, 1), # [640, 8, 8]
            nn.BatchNorm2d(640),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_layer_p),
            
            nn.Conv2d(640, 768, 3, 1, 1), # [768, 8, 8]
            nn.BatchNorm2d(768),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_layer_p),
            
            nn.Conv2d(768, 896, 3, 1, 1), # [768, 8, 8]
            nn.BatchNorm2d(896),
            nn.ReLU(),
            nn.Dropout2d(p=dropout_layer_p),
            
            
            nn.Conv2d(896, 896, 3, 1, 1), # [640, 8, 8]
            nn.BatchNorm2d(896),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [640, 4, 4]
            nn.Dropout2d(p=dropout_layer_p)
        )
        self.fc = nn.Sequential(
            nn.Linear(896*4*4, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        
        out = self.cnn(x)
        
        out = out.view(out.size()[0], -1)
        return self.fc(out)

### Configurations

In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Times of cross Validation
cross = 2

# The number of training epochs.
n_epochs = 150

# Initialize a model, and put it on the device specified.
# model = Classifier().to(device)
model = []
optimizer = []
for i in range(cross):
    model.append(Classifier().to(device))
    # num_ftrs = model[i].fc.in_features
    # model[i].fc = nn.Linear(num_ftrs, 11)
    # model[i].to(device)
    # Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
    optimizer.append(torch.optim.Adam(model[i].parameters(), lr=1e-3, weight_decay=1e-4, amsgrad=True))

    
# The number of batch size.
batch_size = 256


# If no improvement in 'patience' epochs, early stop.
patience = 40

# The ratio of test_time_augmentation
test_time_augmentation = 0.8

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

### Dataloader

In [ ]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = FoodDataset("/kaggle/input/ml2023spring-hw3/train", tfm=train_tfm)
#train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_set = FoodDataset("/kaggle/input/ml2023spring-hw3/valid", tfm=test_tfm)
#valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

dataset = ConcatDataset([train_set, valid_set])

# print("nananananasasasasawawawawamimimimimiaaaaa")
train_size = int(0.733 * len(dataset))
test_size = len(dataset) - train_size


### Start Training

In [ ]:
###### Learning rate schedule
'''def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.99 ** epoch)
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew'''

# Initialize trackers, these are not parameters and should not be changed
stale = 0

for i in range(cross):
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
    
    best_acc = 0
    
    ###### Randomly splitting the set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    
    for epoch in range(n_epochs):

        # ---------- Training ----------
        # Make sure the model is in train mode before training.
        model[i].train()

        # These are used to record information in training.
        train_loss = []
        train_accs = []

        for batch in tqdm(train_loader):

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()
            #print(imgs.shape,labels.shape)

            # Forward the data. (Make sure data and model are on the same device.)
            logits = model[i](imgs.to(device))

            # Calculate the cross-entropy loss.
            # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            loss = criterion(logits, labels.to(device))

            # Gradients stored in the parameters in the previous step should be cleared out first.
            optimizer[i].zero_grad()

            # Compute the gradients for parameters.
            loss.backward()

            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model[i].parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer[i].step()

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            train_loss.append(loss.item())
            train_accs.append(acc)

        train_loss = sum(train_loss) / len(train_loss)
        train_acc = sum(train_accs) / len(train_accs)

        # Print the information.
        print(f"[ Train | No.{i+1} loop | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

        # ---------- Validation ----------
        # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
        model[i].eval()

        # These are used to record information in validation.
        valid_loss = []
        valid_accs = []

        # Iterate the validation set by batches.
        for batch in tqdm(valid_loader):

            # A batch consists of image data and corresponding labels.
            imgs, labels = batch
            #imgs = imgs.half()

            # We don't need gradient in validation.
            # Using torch.no_grad() accelerates the forward process.
            with torch.no_grad():
                logits = model[i](imgs.to(device))

            # We can still compute the loss (but not the gradient).
            loss = criterion(logits, labels.to(device))

            # Compute the accuracy for current batch.
            acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

            # Record the loss and accuracy.
            valid_loss.append(loss.item())
            valid_accs.append(acc)
            #break

        # The average loss and accuracy for entire validation set is the average of the recorded values.
        valid_loss = sum(valid_loss) / len(valid_loss)
        valid_acc = sum(valid_accs) / len(valid_accs)

        # Print the information.
        print(f"[ Valid | No.{i+1} loop | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


        # update logs
        if valid_acc > best_acc:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best")
        else:
            with open(f"./{_exp_name}_log.txt","a"):
                print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


        # save models
        if valid_acc > best_acc:
            print(f"Best model {i+1} found at epoch {epoch+1}, saving model")
            torch.save(model[i].state_dict(), f"{_exp_name}_best{i}.ckpt") # only save best to prevent output memory exceed error
            best_acc = valid_acc
            stale = 0
        else:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break

### Dataloader for test

In [ ]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_set = FoodDataset("/kaggle/input/ml2023spring-hw3/test", tfm=test_tfm)
test_set1 = FoodDataset("/kaggle/input/ml2023spring-hw3/test", tfm=train_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
test_loader1 = DataLoader(test_set1, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

### Testing and generate prediction CSV

In [ ]:
model_best = []
for i in range(cross):
    model_best.append(Classifier().to(device))
    #num_ftrs = model_best[i].fc.in_features
    #model_best[i].fc = nn.Linear(num_ftrs, 11)
    #model_best[i].to(device)
    model_best[i].load_state_dict(torch.load(f"{_exp_name}_best{i}.ckpt"))
    model_best[i].eval()

prediction = []
prediction0 = []
prediction1 = []

with torch.no_grad():
    
    for k in range(cross):
        i = 0
        for data,_ in tqdm(test_loader):
            test_pred = model_best[k](data.to(device))
            if k != 0:
                prediction0[i] = np.add( prediction0[i] , test_pred.cpu().data.numpy())
                i += 1
            else:
                prediction0.append(test_pred.cpu().data.numpy())
            #test_label = test_time_augmentation * np.argmax(test_pred.cpu().data.numpy(), axis=1)
            #prediction += test_label.squeeze().tolist()
    prediction0 = [ i / cross for i in prediction0 ]
        
with torch.no_grad():
    for k in range(cross):
        i = 0
        for data,_ in tqdm(test_loader1):
            test_pred = model_best[k](data.to(device))
            if k != 0:
                prediction1[i] = np.add( prediction1[i] , test_pred.cpu().data.numpy())
                i += 1
            else:
                prediction1.append(test_pred.cpu().data.numpy())
            
    prediction1 = [ i / cross for i in prediction1 ]
    
 
        #test_label = (1 - test_time_augmentation) * np.argmax(test_pred.cpu().data.numpy(), axis=1)
        #prediction1 += test_label.squeeze().tolist()

for i in range(len(prediction0)):
    prediction0[i] = np.add(test_time_augmentation * prediction0[i],(1-test_time_augmentation) * prediction1[i])
    test_label = np.argmax(prediction0[i], axis=1)
    prediction += test_label.squeeze().tolist()

###### Test time augmentation
# prediction = np.around(np.sum([prediction,prediction1],axis=0).tolist()).astype(int)

#print(len(prediction))

In [ ]:
# create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

## Q1. Augmentation Implementation
### Implement augmentation by finishing train_tfm in the code with image size of your choice. 
### Directly copy the following block and paste it on GradeScope after you finish the code
#### Your train_tfm must be capable of producing 5+ different results when given an identical image multiple times.
#### Your  train_tfm in the report can be different from train_tfm in your training code.

In [ ]:
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You can add some transforms here.
    #
    tfm_p = 0.1,
    # Adjust the sharpness of the image randomly with a given probability.
    # 0 gives a blurred image. 1,2是增加敏銳度，我選讓圖片模糊是覺得這樣跟原圖會比較不同
    transforms.RandomAdjustSharpness(sharpness_factor=0, p=tfm_p),
    
    # Equalize the histogram of the given image randomly with a given probability.
    transforms.RandomEqualize(p=tfm_p),
    
    #會讓圖片看起來某一邊離比較近，一邊比較遠(會呈現梯形)
    transforms.RandomPerspective(p=tfm_p),
    
    #會讓圖片上下顛倒
    transforms.RandomVerticalFlip(p=tfm_p),
    
    #會讓圖片左右顛倒
    transforms.RandomHorizontalFlip(p=tfm_p),
    
    transforms.ToTensor(),
])

## Q2. Visual Representations Implementation
### Visualize the learned visual representations of the CNN model on the validation set by implementing t-SNE (t-distributed Stochastic Neighbor Embedding) on the output of both top & mid layers (You need to submit 2 images). 
### ChatGPT has generated the following code, which requires a minor modification to produce the expected results.

In [ ]:
'''import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from tqdm import tqdm
import matplotlib.cm as cm
import torch.nn as nn

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load the trained model
model = Classifier().to(device)
state_dict = torch.load(f"{_exp_name}_best.ckpt")
model.load_state_dict(state_dict)
model.eval()

print(model)'''

In [ ]:
'''# Load the vaildation set defined by TA
valid_set = FoodDataset("/kaggle/input/ml2023spring-hw3/valid", tfm=test_tfm)
valid_loader = DataLoader(valid_set, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

# Extract the representations for the specific layer of model
index = 14 # You should find out the index of layer which is defined as "top" or 'mid' layer of your model.
features = []
labels = []
for batch in tqdm(valid_loader):
    imgs, lbls = batch
    with torch.no_grad():
        logits = model.cnn[:index](imgs.to(device))
        logits = logits.view(logits.size()[0], -1)
    labels.extend(lbls.cpu().numpy())
    logits = np.squeeze(logits.cpu().numpy())
    features.extend(logits)
    
features = np.array(features)
colors_per_class = cm.rainbow(np.linspace(0, 1, 11))

# Apply t-SNE to the features
features_tsne = TSNE(n_components=2, init='pca', random_state=42).fit_transform(features)

# Plot the t-SNE visualization
plt.figure(figsize=(10, 8))
for label in np.unique(labels):
    plt.scatter(features_tsne[labels == label, 0], features_tsne[labels == label, 1], label=label, s=5)
plt.legend()
plt.show()'''